In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [52]:
pulsar_data = pd.read_csv('pulsar_stars.csv', delimiter=',')
cols = pd.read_csv('pulsar_stars.csv', nrows=1).columns

#print(pulsar_data.describe())
dummy_ranks = pd.get_dummies(pulsar_data['target_class'],prefix='target_class')
print(dummy_ranks.head())

pulsar_data['intercept'] = 1.0

   target_class_0  target_class_1
0               1               0
1               1               0
2               1               0
3               1               0
4               1               0


In [56]:
cols_to_keep = cols.values[:-1]
data = pulsar_data[cols_to_keep].join(dummy_ranks.loc[:, 'target_class_1'])
train_cols = data.columns[:-1]
#print(train_cols)
logit = sm.Logit(pulsar_data['target_class'], pulsar_data[train_cols])
result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.075602
         Iterations 9


In [59]:
print(result.summary())
print(result.conf_int())

                           Logit Regression Results                           
Dep. Variable:           target_class   No. Observations:                17898
Model:                          Logit   Df Residuals:                    17890
Method:                           MLE   Df Model:                            7
Date:                Tue, 24 Sep 2019   Pseudo R-squ.:                  0.7531
Time:                        20:55:19   Log-Likelihood:                -1353.1
converged:                       True   LL-Null:                       -5479.8
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                                    coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
 Mean of the integrated profile                  -0.0013      0.005     -0.259      0.796      -0.011       0.008
 Standard deviation of the